In [ ]:
# Nota: este notebook no se puede ejecutar a menos que hayas corrido el archivo "ProyectoJuegosMesa.ipynb" y hayas generado los archivos "test.csv" y "train.csv"
import pyreclab
import random
import pandas as pd
from surprise import accuracy

top_n = 10

In [ ]:
# Evaluamos SVD
mySVD = pyreclab.SVD(dataset='train.csv', dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1)
mySVD.train(factors=50, maxiter=80, lr=0.1, lamb=0.5)
_, maeSVD, rmseSVD = mySVD.test(input_file = 'test.csv', dlmchar = b',', header = False, usercol = 2, itemcol = 0, ratingcol = 1)
_, mapSVD, ndcgSVD = mySVD.testrec(input_file="test.csv", dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1, topn=top_n)

print(f"mae = {maeSVD} y rmse = {rmseSVD}")
print(f"map = {mapSVD} y ndcg = {ndcgSVD}")

In [ ]:
# Entrenamos UserKnn
myUserKnn = pyreclab.UserKnn(dataset='train.csv', dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1)
myUserKnn.train(k=7, similarity='pearson')

# Evaluamos UserKNN
_, maeUK, rmseUK = myUserKnn.test(input_file = 'test.csv', dlmchar = b',', header = False, usercol = 2, itemcol = 0, ratingcol = 1)
_, mapUK, ndcgUK = myUserKnn.testrec(input_file="test.csv", dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1, topn=top_n)

print(f"mae = {maeUK} y rmse = {rmseUK}")
print(f"map = {mapUK} y ndcg = {ndcgUK}")

In [ ]:
# Entrenamos ItemKnn
myItemKnn = pyreclab.ItemKnn(dataset='train.csv', dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1)
myItemKnn.train(k=7, similarity='pearson')

_, maeIK, rmseIK = myItemKnn.test(input_file = 'test.csv', dlmchar = b',', header = False, usercol = 2, itemcol = 0, ratingcol = 1)
_, mapIK, ndcgIK = myItemKnn.testrec(input_file="test.csv", dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1, topn=top_n)

print(f"mae = {maeIK} y rmse = {rmseIK}")
print(f"map = {mapIK} y ndcg = {ndcgIK}")

In [ ]:
# Evaluamos Most Popular
myMP = pyreclab.MostPopular(dataset='train.csv', dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1)
myMP.train(progress=False)
_, mapMP, ndcgMP = myMP.testrec(input_file="test.csv", dlmchar=b',', header=False, usercol=2, itemcol=0, ratingcol=1, topn=top_n)

print(f"map = {mapMP} y ndcg = {ndcgMP}")

In [ ]:
# Evaluamos Random ratings
predictions = []

rating_scale = (1, 10)

test_set = pd.read_csv('test.csv')
test_set['Rating'] = test_set['Rating'].astype(float)

test_set.head(top_n)
for _, row in test_set.iterrows():
    itemId = row["BGGId"]; rating = row["Rating"]; userId = row["Username"]
    random_rating = random.uniform(rating_scale[0], rating_scale[1])
    predictions.append((userId, itemId, rating, random_rating, None))

accuracy.rmse(predictions)
accuracy.mae(predictions)